In [2]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np

In [ ]:
column_of_interest = ["text_ tokens"]
train_set = pd.read_csv('s3://recsys-challenge-2020/train_set.csv', encoding="utf-8",
                     usecols= [1])

In [ ]:
column_of_interest = ["text_ tokens", "engaging_user_id"]
train_set = pd.read_csv('s3://recsys-challenge-2020/train_set.csv', encoding="utf-8",
                     usecols= [1, 4])

In [ ]:
train_set.head()

In [ ]:
column_of_interest = ["text_ tokens", "engaging_user_id"]
val_set = pd.read_csv('s3://recsys-challenge-2020/val_set.csv', encoding="utf-8",
                     usecols= [1, 4])

In [ ]:
unique_val_set_users = val_set[['engaging_user_id']].drop_duplicates()

In [ ]:
warm_train_set = pd.merge(train_set, unique_val_set_users, how = 'inner', on = 'engaging_user_id')

In [ ]:
warm_train_set.head()

In [ ]:
warm_train_set.to_csv('s3://recsys-challenge-2020/warm_train_set.csv', index = False)

In [3]:
warm_train_set = pd.read_csv('s3://recsys-challenge-2020/warm_train_set.csv', encoding="utf-8")

In [4]:
len(warm_train_set)

62130438

In [5]:
warm_train_set

,text_ tokens,engaging_user_id
0,101\t108\t31278\t90939\t70325\t196\t199\t71436...,12A1AF0088C5B4FCBFA024D4A1971323
1,101\t56898\t137\t10173\t19402\t168\t57923\t594...,12A1AF0088C5B4FCBFA024D4A1971323
2,101\t10657\t10196\t100\t20020\t49533\t10127\t3...,12A1AF0088C5B4FCBFA024D4A1971323
3,101\t56898\t137\t12042\t12419\t10457\t131\t100...,12A1AF0088C5B4FCBFA024D4A1971323
4,101\t10117\t12518\t45415\t10472\t10114\t19072\...,77309DE08AB631D8224254D33648F287
...,...,...
62130433,101\t144\t18913\t13998\t17532\t10115\t12862\t1...,BC88789FD751A1929D1CB7786E7C1619
62130434,101\t1894\t4423\t2490\t5610\t1981\t104928\t326...,F2758BCA710E4F1A0AFB9976D31B4599
62130435,101\t2179\t1980\t35412\t74381\t11312\t29442\t1...,27DE351CB0C5EC4649D28539A48166D9
62130436,101\t14265\t50101\t49362\t895\t13718\t896\t174...,76C665A821863701543C0346E1DBBB24


In [6]:
user_tokens = warm_train_set.groupby(['engaging_user_id']).agg(list)

In [7]:
user_tokens.head()

,text_ tokens
engaging_user_id,
000000E4C259FAD27812910B1DF398D7,[101\t56898\t137\t38516\t14013\t73477\t16105\t...
000006829BEADA9EEA695CF0C334B426,[101\t2435\t5656\t2594\t8279\t8623\t1925\t6412...
00000B85AAF7DE172876FD96718C4469,[101\t56898\t137\t34372\t11043\t168\t188\t1514...
00000D0F0E3839F5D001C77689B36182,[101\t28248\t20706\t117\t10163\t93568\t29956\t...
00000DEF82BE9EB5CFD07FB7DB94317B,[101\t108\t50884\t71655\t10731\t119\t119\t8308...


In [8]:
len(user_tokens)

9830047

In [ ]:
len(train_set)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [ ]:
iterator = 0
for chunk in np.array_split(warm_train_set, 10000):
    print(iterator)
    iterator = iterator + 1
    df_embeddings = pd.DataFrame()
    df_embeddings = chunk[["engaging_user_id"]]
    df_embeddings['text_embeddings'] = chunk['text_ tokens'].\
apply(lambda x : model(torch.tensor(list(map(int, x.split('\t')))).unsqueeze(0))[0][0][0])
    df_embeddings.to_csv('s3://recsys-challenge-2020/embeddings_user.csv', mode='a', header=False)

In [ ]:
iterator = 0
for chunk in np.array_split(train_set, 10000):
    print(iterator)
    iterator = iterator + 1
    df_embeddings = pd.DataFrame()
    df_embeddings['text_embeddings'] = chunk['text_ tokens'].\
apply(lambda x : model(torch.tensor(list(map(int, x.split('\t')))).unsqueeze(0))[0][0][0])
    df_embeddings.to_csv('s3://recsys-challenge-2020/embeddings.csv', mode='a', header=False)